In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import json 
import os

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh;Intel Mac OS X 10_15_4)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36',
}


In [ ]:
def getgamelist(n):
    linklist=[]
    IDlist = []
    for pagenum in range(1,n):
        try:
            r = requests.get('https://store.steampowered.com/search/?ignore_preferences=1&category1=998&os=win&filter=globaltopsellers&page=%d'%pagenum,headers=headers)
        except:
            print('timeout')
            continue
        soup = BeautifulSoup(r.text, 'lxml')
        soups= soup.find_all(href=re.compile(r"https://store.steampowered.com/app/"),class_="search_result_row ds_collapse_flag")
        for i in soups:
            i = i.attrs
            i = i['href']
            link = re.search('https://store.steampowered.com/app/(\d*?)/',i).group()
            ID = re.search('https://store.steampowered.com/app/(\d*?)/(.*?)/', i).group(1)
            linklist.append(link)
            IDlist.append(ID)
        print('已完成'+str(pagenum)+'页,目前共'+str(len(linklist)))
    return linklist,IDlist

def getdf(n):#转df
    linklist,IDlist = getgamelist(n)
    df = pd.DataFrame(list(zip(linklist,IDlist)),
               columns =['Link', 'ID'])
    return df

path = 'url.csv'
if os.path.isfile(path):
    df = pd.read_csv(path)
    print(df.tail(5))
else:
    df = getdf(20)
    df.to_csv(path, index = False)


                                            Link       ID
458  https://store.steampowered.com/app/1158980/  1158980
459  https://store.steampowered.com/app/1222700/  1222700
460  https://store.steampowered.com/app/1307890/  1307890
461  https://store.steampowered.com/app/1173790/  1173790
462  https://store.steampowered.com/app/1113000/  1113000


In [ ]:
def gamename(soup):   #游戏名字
    try:
        a = soup.find(class_="apphub_AppName")
        k = str(a.string)
    except:
        a = soup.find(class_="apphub_AppName")
        k = str(a.text)
    return k

def gameprice(soup):#价格
    try:
        a = soup.findAll(class_="discount_original_price")
        for i in a:
            if re.search('$|¥|free|免费', str(i),re.IGNORECASE):
                a = i
        k = str(a.string).replace('	', '').replace('\n', '').replace('\r', '').replace(' ', '')
    except:
        a = soup.findAll(class_="game_purchase_price price")
        for i in a:
            if re.search('$|¥|free|免费', str(i),re.IGNORECASE):
                a = i
        k = str(a.string).replace('	', '').replace('\n', '').replace('\r', '').replace(' ', '')
    return k

def taglist(soup):#标签列表
    list1=[]
    a = soup.find_all(class_="app_tag")
    for i in a:
        k = str(i.string).replace('	', '').replace('\n', '').replace('\r', '')
        if k == '+':
            pass
        else:
            list1.append(k)
    list1 = str('\n'.join(list1))
    return list1

def description(soup):  #游戏描述
    a = soup.find(class_="game_description_snippet")
    k = str(a.string).replace('	', '').replace('\n', '').replace('\r', '')
    return k

def reviewsummary(soup):   #总体评价
    a = soup.find(class_="summary column")
    try:
        k = str(a.span.string)
    except:
        k=str(a.text)
    return k

def getdate(soup):   #发行日期
    a = soup.find(class_="date")
    k = str(a.string)
    return k

def userreviewsrate(soup):#总体数量好评率
    a = soup.find(class_="user_reviews_summary_row")
    k = str((a.attrs)['data-tooltip-html'])
    return k

def developer(soup):   #开发商
    a = soup.find(id="developers_list")
    k = str(a.a.string)
    return k

def getreviews(ID):#获取评论
    r1 = requests.get(
        'https://store.steampowered.com/appreviews/%s?cursor=*&day_range=30&start_date=-1&end_date=-1&date_range_type=all&filter=summary&language=english&l=english&review_type=all&purchase_type=all&playtime_filter_min=0&playtime_filter_max=0&filter_offtopic_activity=1'%str(ID),headers=headers,timeout=10)

    soup = BeautifulSoup(r1.json()['html'], 'lxml')
    a = soup.findAll(class_="content")
    list1 = []
    count=0
    for i in a:
        count+=1
        list1.append(i.text.replace('	', '').replace('\n', '').replace('\r', ''))
        if count>10:
            break
    
    k=str('\n'.join(list1))
    return k

def getdetail(x):
    tag, des, reviews, date, rate, dev, review,name,price = ' ', ' ', ' ', ' ', ' ', ' ', ' ',' ',' '
    global count
    try:
        r = requests.get(x['Link'], headers=headers,timeout=10)
    except:
        print('服务器无响应1',end=' ')
        try:
            r = requests.get(x['Link'], headers=headers,timeout=10)
        except:
            print('服务器无响应2',end=' ')
            try:
                r = requests.get(x['Link'], headers=headers,timeout=10)
            except:
                print('服务器无响应3',end=' ')

    price = -1

    try:
        soup = BeautifulSoup(r.text, 'lxml')
        name = gamename(soup)
        tag = taglist(soup)
        des = description(soup)
        reviews = reviewsummary(soup)
        date = getdate(soup)
        rate = userreviewsrate(soup)
        dev = developer(soup)
        review = getreviews(str(x['ID']))
        price = gameprice(soup)
        print(count,'y',end=' ')
        # print('已完成: '+name+str(x['ID'])+'第%d个'%count,end='')
    except:
        print(count,'n',end=' ')
        # print('未完成:  '+str(x['ID'])+'第%d个'%count,end='')


    count += 1
    return name,price,tag,des,reviews,date,rate,dev,review


path = 'detail.csv'
df1 = df
if os.path.isfile(path):
    df1 = pd.read_csv(path)
    print(df1.tail(5))
else:
    count = 1
    df1['详细'] = df1.apply(lambda x: getdetail(x), axis=1)
    df1['名字'] = df1.apply(lambda x: x['详细'][0], axis=1)
    df1['价格'] = df1.apply(lambda x: x['详细'][1], axis=1)
    df1['标签'] = df1.apply(lambda x: x['详细'][2], axis=1)
    df1['描述'] = df1.apply(lambda x: x['详细'][3], axis=1)
    df1['近期评价'] = df1.apply(lambda x: x['详细'][4], axis=1)
    df1['发行日期'] = df1.apply(lambda x: x['详细'][5], axis=1)
    df1['近期数量好评率'] = df1.apply(lambda x: x['详细'][6], axis=1)
    df1['开发商'] = df1.apply(lambda x: x['详细'][7], axis=1)
    df1['评论'] = df1.apply(lambda x: x['详细'][8], axis=1)
    df1.to_csv(path, index = False)




                                            Link  ...                                                 评论
458  https://store.steampowered.com/app/1158980/  ...  It's really not an idle game since you have to...
459  https://store.steampowered.com/app/1222700/  ...  ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣠⠞⠉⢉⠩⢍⡙⠛⠋⣉⠉⠍...
460  https://store.steampowered.com/app/1307890/  ...  TL;DR: rough around the edges, solid colony bu...
461  https://store.steampowered.com/app/1173790/  ...  I've had the opportunity to say this on other ...
462  https://store.steampowered.com/app/1113000/  ...  friend simulator :)\nThe best way to experienc...

[5 rows x 12 columns]


In [ ]:
df2 = df1.copy()
df2.iloc[23]

Link             https://store.steampowered.com/app/1178830/
ID                                                   1178830
详细         ('Bright Memory: Infinite', 'NT$281', 'Action\...
名字                                   Bright Memory: Infinite
价格                                                    NT$281
标签         Action\nIndie\nFPS\nFemale Protagonist\nSci-fi...
描述         Bright Memory: Infinite is an all-new lightnin...
近期评价                                           Very Positive
发行日期                                            11 Nov, 2021
近期数量好评率    92% of the 10,830 user reviews for this game a...
开发商                                              FYQD-Studio
评论         ⚖️ Grade = B. Worth a buy, if you enjoy short ...
Name: 23, dtype: object

In [ ]:
# game
#   id
#   name
#   price
#   description
#   release_date
#   status
df2 = df1.copy()
df3 = df2.rename(columns={"ID": "id", "名字": "name", "价格": "price", "描述": "description", "发行日期": "release_date"})

def price_process(p):
  s = str(p).lower().replace('freetoplay','0').replace('nt$','').replace(',','')
  try:
    return int(s)//20
  except:
    return 0

def date_process(da):
  dic = {'jan':1,'feb':2,'mar':3,'apr':4,'may':5,'jun':6,'jul':7,'aug':8,'sep':9,'oct':10,'nov':11,'dec':12}
  arr = da.strip().split()[::-1]
  try:
    day =  arr[0] + '-' + str(dic[arr[1].lower().replace(',','')]).rjust(2,'0') + '-' + (arr[2] if len(arr)==3 else '11')
    day = day if len(day) == 10 else 2011-11-11
    return day + ' 11:11:11'
  except:
    return '2011-11-11 11:11:11'

df3['price'] = df3.apply(lambda x: price_process(x['price']), axis=1)
df3['release_date'] = df3.apply(lambda x: date_process(x['release_date']), axis=1)
df3['status'] = 1
game_df = df3[["id","name","price","description","release_date","status"]]
game_df.to_csv('game.csv', index = False)
df3[["id","name","price","description","release_date","status"]].tail(5)



,id,name,price,description,release_date,status
458,1158980,Idle Big Devil,14,Idle Big Devil is a free idle RPG game with 3D...,2011-11-11 11:11:11,1
459,1222700,A Way Out,39,A Way Out is an exclusively co-op adventure wh...,2018-03-23 11:11:11,1
460,1307890,Kingdoms Reborn,15,"A city builder with simulated citizens, set in...",2020-11-11 11:11:11,1
461,1173790,FINAL FANTASY III,24,The original FINAL FANTASY III comes to life w...,2021-07-28 11:11:11,1
462,1113000,Persona 4 Golden,39,A coming of age story that sets the protagonis...,2020-06-13 11:11:11,1


In [ ]:
# comment
#   id
#   content
#   rating
#   game_id
#   user_id
import random
def f(s):
    return s.text.replace('	', '').replace('\n', '').replace('\r', '')

def reviews(ID):#获取评论
    def rand_rating(rating):
        if len(rating.strip()) < 13 :
            return random.randint(1,3)
        else:
            return random.randint(3,5)
    global cmt_id
    r1 = requests.get(
        'https://store.steampowered.com/appreviews/%s?cursor=*&day_range=30&start_date=-1&end_date=-1&date_range_type=all&filter=summary&language=english&l=english&review_type=all&purchase_type=all&playtime_filter_min=0&playtime_filter_max=0&filter_offtopic_activity=1'%str(ID),headers=headers,timeout=10)

    soup = BeautifulSoup(r1.json()['html'], 'lxml')
    users = soup.findAll(class_="persona_name")
    contents = soup.findAll(class_="content")
    ratings = soup.findAll(class_="title ellipsis")

    for i in range( min(len(users), len(contents), len(ratings), 10)):
        cmt_df.loc[cmt_id] = [str(cmt_id), f(contents[i]), str(rand_rating(f(ratings[i]))), ID, -1, f(users[i])]
        cmt_id += 1

path = 'comment_data.csv'
if os.path.isfile(path):
    cmt_df = pd.read_csv(path)
    print(cmt_df.iloc[0])
else:
    cmt_id = 0
    cmt_df = pd.DataFrame({x :[] for x in ["id", "content", "rating", "game_id", "user_id", "user_name"]})
    for i, id in enumerate(df['ID']):
        print(i, end=' ')
        reviews(str(id))
    cmt_df.to_csv(path, index = False)


0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 

KeyboardInterrupt: ignored

In [ ]:
# user(comment)
#   id
#   username
#   password
#   role
user2id, uid = {'admin':0}, 1
for user_name in cmt_df["user_name"]:
    if user_name not in user2id:
        user2id[user_name] = uid
        uid += 1

fin_cmt_df = cmt_df.copy()
fin_cmt_df['user_id'] = fin_cmt_df.apply(lambda x: user2id[x['user_name']], axis=1)
print(fin_cmt_df.head(1))
fin_cmt_df[["id", "content", "rating", "game_id", "user_id"]].to_csv('comment.csv', index = False)


user_df = fin_cmt_df.copy()[["user_id", "user_name"]].drop_duplicates().rename(columns={"user_name": "username", "user_id":"id"})
user_df = user_df.append({'username': 'admin', 'id':str(0)}, ignore_index=True)
user_df['password'] = user_df.apply(lambda x: x['username'], axis=1)
user_df['role'] = 'user'
user_df.loc[user_df.index[-1], 'role']= 'admin'

print(user_df.tail(1))
user_df.to_csv('user.csv', index = False)



  id                                            content  ... user_id  user_name
0  0  What do you call a Mexican who lost his car? C...  ...       1  buru nyuu

[1 rows x 6 columns]
     id username password   role
4258  0    admin    admin  admin


In [ ]:
# game_order      
#   order_id
#   game_id

# order
#   id
#   amount
#   create_time
#   user_id

order_df = fin_cmt_df[['id','game_id', 'user_id']].copy()
order_df['create_time'] = '2011-11-11 11:11:11'
order_df['amount'] = 1

order_df
order_df.rename(columns={"id": "order_id"})[['order_id','game_id']].to_csv('game_order.csv', index = False)
order_df[['id','amount','create_time','user_id']].to_csv('order.csv', index = False)

In [ ]:
# game_tag
#   game_id
#   tag_name


def get_tags(ID, link):
    global tag_df
    def f(s):
        return s.text.replace('	', '').replace('\n', '').replace('\r', '')
    r1 = requests.get(link,headers=headers,timeout=10)

    soup = BeautifulSoup(r1.text, 'lxml')
    tags = soup.findAll(class_="app_tag")
    for i in range(min(len(tags), 6)):
        tag_df = tag_df.append({'game_id': ID, 'tag_name':f(tags[i])}, ignore_index=True)

tag_df = pd.DataFrame({x :[] for x in ["game_id", "tag_name"]})
for index, row in df1.iterrows():
    print(index, end = ' ')
    get_tags(str(row['ID']), row['Link'])

tag_df.to_csv('game_tag.csv', index = False)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [ ]:
# picture
#   id
#   picture_url 
#   game_id


def get_img(ID, link):
    global img_df
    r1 = requests.get(link,headers=headers,timeout=10)
    soup = BeautifulSoup(r1.text, 'lxml')
    
    thumb = soup.find(class_="game_header_image_full")
    tags = soup.findAll(class_="highlight_strip_item highlight_strip_screenshot")

    try:
        img_df = img_df.append({'game_id': ID, 'picture_link':thumb["src"]}, ignore_index=True)
    except:
        img_df = img_df.append({'game_id': ID, 'picture_link':'https://cdn.akamai.steamstatic.com/steam/apps/1517290/header.jpg'}, ignore_index=True)

    if len(tags) == 0:
        img_df = img_df.append({'game_id': ID, 'picture_link':'https://cdn.akamai.steamstatic.com/steam/apps/1517290/header.jpg'}, ignore_index=True)
    for i in range(min(len(tags), 4)):
        img_df = img_df.append({'game_id': ID, 'picture_link':tags[i].img["src"]}, ignore_index=True)
    
    

img_df = pd.DataFrame({x :[] for x in ["game_id", "picture_link"]})
for index, row in df.iterrows():
    print(index, end = ' ')
    get_img(str(row['ID']), row['Link'])

img_df.to_csv('img_url.csv', index = False)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [ ]:
import base64
def url2base(url):
    return base64.b64encode(requests.get(url).content)

img_fin_df = img_df.copy()
img_fin_df['picture_url'] = img_fin_df.apply(lambda x: url2base(x['picture_link']), axis=1)

In [ ]:
img_fin_df
img_fin_df[['game_id', 'picture_url']].to_csv('picture.csv')

# for i in range(len(img_fin_df)):
#     if len(img_fin_df.iloc[i]['picture_url'])<10:
#         print('?')

In [ ]:
img_fin_df

,game_id,picture_link,picture_url
0,1517290,https://cdn.akamai.steamstatic.com/steam/apps/...,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAMCAgMCAgMD...
1,1517290,https://cdn.akamai.steamstatic.com/steam/apps/...,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAMCAgMCAgMD...
2,1551360,https://cdn.cloudflare.steamstatic.com/steam/a...,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAMCAgMCAgMD...
3,1551360,https://cdn.cloudflare.steamstatic.com/steam/a...,b'/9j/4AAQSkZJRgABAQEAeAB4AAD/2wBDAAMCAgMCAgMD...
4,1551360,https://cdn.cloudflare.steamstatic.com/steam/a...,b'/9j/4AAQSkZJRgABAQEAeAB4AAD/2wBDAAMCAgMCAgMD...
...,...,...,...
2022,1173790,https://cdn.cloudflare.steamstatic.com/steam/a...,b'/9j/4AAQSkZJRgABAQIAHAAcAAD/2wBDAAMCAgMCAgMD...
2023,1173790,https://cdn.cloudflare.steamstatic.com/steam/a...,b'/9j/4AAQSkZJRgABAQIAHAAcAAD/2wBDAAMCAgMCAgMD...
2024,1173790,https://cdn.cloudflare.steamstatic.com/steam/a...,b'/9j/4AAQSkZJRgABAQIAHAAcAAD/2wBDAAMCAgMCAgMD...
2025,1113000,https://cdn.akamai.steamstatic.com/steam/apps/...,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAMCAgMCAgMD...
